In [0]:
storage_account_key = "nFNO4+sXSRdXhPWwkcRvuocK7MBQMDqK1QPn0EwIAz0d6Pr0TIGzcB3b2RjGVgq45B+Z8KV9bQuW+AStpfqh4Q=="
dbutils.fs.mount(
    source="wasbs://bronze@hoangtien.blob.core.windows.net",
    mount_point="/mnt/lakehouse1/bronze",
    extra_configs={
        "fs.azure.account.key.hoangtien.blob.core.windows.net": storage_account_key
    },
)

In [0]:
storage_account_key = "nFNO4+sXSRdXhPWwkcRvuocK7MBQMDqK1QPn0EwIAz0d6Pr0TIGzcB3b2RjGVgq45B+Z8KV9bQuW+AStpfqh4Q=="
dbutils.fs.mount(
    source="wasbs://silver@hoangtien.blob.core.windows.net",
    mount_point="/mnt/lakehouse1/silver",
    extra_configs={
        "fs.azure.account.key.hoangtien.blob.core.windows.net": storage_account_key
    },
)

In [0]:
storage_account_key = "nFNO4+sXSRdXhPWwkcRvuocK7MBQMDqK1QPn0EwIAz0d6Pr0TIGzcB3b2RjGVgq45B+Z8KV9bQuW+AStpfqh4Q=="
dbutils.fs.mount(
    source="wasbs://gold@hoangtien.blob.core.windows.net",
    mount_point="/mnt/lakehouse1/gold",
    extra_configs={
        "fs.azure.account.key.hoangtien.blob.core.windows.net": storage_account_key
    },
)

In [0]:
# 1. Thông tin Event Hub
connectionString = "Endpoint=sb://ehnamespace-kafka.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=c2nNhCQl+5k70RZ2XhsvjDdh18g1kmtyM+AEhEh6/Dk=;EntityPath=myeventhub"

# 2. Cấu hình kết nối (phải encrypt)
ehConf = {
  'eventhubs.connectionString': sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)
}

# 3. Định nghĩa schema cho JSON
from pyspark.sql.types import StructType, StringType
from pyspark.sql.functions import from_json, col

from pyspark.sql.types import *

schema = StructType([
    StructField("city_name", StringType()),
    StructField("timestamp", StringType()),
    StructField("name", StringType()),  # Tên trong dữ liệu gốc (ví dụ: Huế)
    StructField("weather", ArrayType(StructType([
        StructField("main", StringType()),
        StructField("description", StringType())
    ]))),
    StructField("main", StructType([
        StructField("temp", FloatType()),
        StructField("feels_like", FloatType()),
        StructField("humidity", IntegerType()),
        StructField("pressure", IntegerType())
    ])),
    StructField("wind", StructType([
        StructField("speed", FloatType())
    ])),
    StructField("clouds", StructType([
        StructField("all", IntegerType())
    ])),
    StructField("sys", StructType([
        StructField("country", StringType())
    ]))
])


# 4. Đọc từ Event Hub (streaming)
df_raw = (
    spark.readStream
         .format("eventhubs")
         .options(**ehConf)
         .load()
)

# 5. Parse nội dung JSON
from pyspark.sql.functions import from_json, col, expr

df_parsed = (
    df_raw.selectExpr("CAST(body AS STRING) as json_str")
          .select(from_json(col("json_str"), schema).alias("data"))
          .select(
              col("data.city_name"),
              col("data.timestamp"),
              col("data.name").alias("location_name"),
              col("data.weather")[0]["main"].alias("weather_main"),
              col("data.weather")[0]["description"].alias("weather_description"),
              col("data.main.temp"),
              col("data.main.feels_like"),
              col("data.main.humidity"),
              col("data.main.pressure"),
              col("data.wind.speed").alias("wind_speed"),
              col("data.clouds.all").alias("cloud_coverage"),
              col("data.sys.country").alias("country_code")
          )
)


# 6. Ghi xuống lớp bronze
(
    df_parsed.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", "/mnt/lakehouse1/checkpoints/eventhub_bronze")
    .option("mergeSchema", "true")
    .start("/mnt/lakehouse1/bronze/eventhub")
)


In [0]:
spark.catalog.clearCache()
df = spark.read.format("delta").load("/mnt/lakehouse1/bronze/eventhub")
df.show()

+---------+--------------------+--------------------+------------+-------------------+-----+----------+--------+--------+----------+--------------+------------+
|city_name|           timestamp|       location_name|weather_main|weather_description| temp|feels_like|humidity|pressure|wind_speed|cloud_coverage|country_code|
+---------+--------------------+--------------------+------------+-------------------+-----+----------+--------+--------+----------+--------------+------------+
|HoChiMinh|2025-05-24T20:37:...|Thành phố Hồ Chí ...|       Clear|bầu trời quang đãng|27.21|     30.43|      82|    1007|      1.03|             0|          VN|
|    Hanoi|2025-05-24T20:38:...|             Xom Pho|      Clouds|       mây đen u ám|24.98|     25.57|      78|    1013|       3.3|           100|          VN|
|   Danang|2025-05-24T20:38:...|               Ap Ba|      Clouds|           mây thưa|28.99|     35.61|      84|    1010|       3.6|            20|          VN|
|    Hanoi|2025-05-24T20:39:...|  

In [0]:
spark.catalog.clearCache()
df_bronze = spark.read.format("delta").load("/mnt/lakehouse1/bronze/eventhub")
df_bronze.show()

+---------+--------------------+--------------------+------------+-------------------+-----+----------+--------+--------+----------+--------------+------------+
|city_name|           timestamp|       location_name|weather_main|weather_description| temp|feels_like|humidity|pressure|wind_speed|cloud_coverage|country_code|
+---------+--------------------+--------------------+------------+-------------------+-----+----------+--------+--------+----------+--------------+------------+
|HoChiMinh|2025-05-24T20:37:...|Thành phố Hồ Chí ...|       Clear|bầu trời quang đãng|27.21|     30.43|      82|    1007|      1.03|             0|          VN|
|    Hanoi|2025-05-24T20:38:...|             Xom Pho|      Clouds|       mây đen u ám|24.98|     25.57|      78|    1013|       3.3|           100|          VN|
|   Danang|2025-05-24T20:38:...|               Ap Ba|      Clouds|           mây thưa|28.99|     35.61|      84|    1010|       3.6|            20|          VN|
|    Hanoi|2025-05-24T20:39:...|  

In [0]:
display(df_bronze)

city_name,timestamp,location_name,weather_main,weather_description,temp,feels_like,humidity,pressure,wind_speed,cloud_coverage,country_code
HoChiMinh,2025-05-25T21:20:07.632504+07:00,Thành phố Hồ Chí Minh,Clouds,mây cụm,27.77,31.39,79,1010,3.6,75,VN
Hanoi,2025-05-25T21:20:08.991564+07:00,Xom Pho,Clouds,mây đen u ám,23.98,24.05,62,1016,5.32,87,VN
Danang,2025-05-25T21:20:10.366949+07:00,Ap Ba,Clouds,mây rải rác,29.0,35.64,84,1011,1.54,40,VN
GiaLai,2025-05-25T21:20:12.354106+07:00,Plây Cu,Clouds,mây đen u ám,21.2,21.86,95,1012,2.63,100,VN
CanTho,2025-05-25T21:20:13.664829+07:00,Cần Thơ,Rain,mưa nhẹ,25.01,26.18,100,1010,0.0,75,VN
Hue,2025-05-25T21:20:15.688052+07:00,Huế,Clouds,mây rải rác,27.05,30.77,89,1011,3.09,40,VN
HoChiMinh,2025-05-25T21:29:13.386561+07:00,Thành phố Hồ Chí Minh,Clouds,mây cụm,28.3,32.52,78,1010,3.6,75,VN
Hanoi,2025-05-25T21:29:15.575476+07:00,Xom Pho,Clouds,mây đen u ám,23.98,24.05,62,1016,5.32,87,VN
Danang,2025-05-25T21:29:17.839396+07:00,Ap Ba,Clouds,mây rải rác,28.99,35.61,84,1011,1.54,40,VN
GiaLai,2025-05-25T21:29:19.850229+07:00,Plây Cu,Clouds,mây đen u ám,21.2,21.86,95,1012,2.63,100,VN


##### 📦 Làm sạch & chuẩn hóa dữ liệu:

In [0]:
from pyspark.sql.functions import to_timestamp, from_utc_timestamp

df_silver = (
    df_bronze
    .withColumn("timestamp", to_timestamp("timestamp"))
    .withColumn("timestamp", from_utc_timestamp(col("timestamp"), "Asia/Ho_Chi_Minh"))
    .withColumnRenamed("location_name", "location")
    .filter(col("location") != "Đà Nẵng")
    .dropna(subset=["city_name", "timestamp", "temp"])
)


##### 💾 Ghi xuống lớp Silver:

In [0]:
df_silver.write.format("delta").mode("overwrite").save("/mnt/lakehouse1/silver/weather")

In [0]:
df_silver.show()

+---------+--------------------+--------------------+------------+-------------------+-----+----------+--------+--------+----------+--------------+------------+
|city_name|           timestamp|            location|weather_main|weather_description| temp|feels_like|humidity|pressure|wind_speed|cloud_coverage|country_code|
+---------+--------------------+--------------------+------------+-------------------+-----+----------+--------+--------+----------+--------------+------------+
|HoChiMinh|2025-05-25 21:20:...|Thành phố Hồ Chí ...|      Clouds|            mây cụm|27.77|     31.39|      79|    1010|       3.6|            75|          VN|
|    Hanoi|2025-05-25 21:20:...|             Xom Pho|      Clouds|       mây đen u ám|23.98|     24.05|      62|    1016|      5.32|            87|          VN|
|   Danang|2025-05-25 21:20:...|               Ap Ba|      Clouds|        mây rải rác| 29.0|     35.64|      84|    1011|      1.54|            40|          VN|
|   GiaLai|2025-05-25 21:20:...|  

In [0]:

display(df_silver)

city_name,timestamp,location,weather_main,weather_description,temp,feels_like,humidity,pressure,wind_speed,cloud_coverage,country_code
HoChiMinh,2025-05-25T21:20:07.632504Z,Thành phố Hồ Chí Minh,Clouds,mây cụm,27.77,31.39,79,1010,3.6,75,VN
Hanoi,2025-05-25T21:20:08.991564Z,Xom Pho,Clouds,mây đen u ám,23.98,24.05,62,1016,5.32,87,VN
Danang,2025-05-25T21:20:10.366949Z,Ap Ba,Clouds,mây rải rác,29.0,35.64,84,1011,1.54,40,VN
GiaLai,2025-05-25T21:20:12.354106Z,Plây Cu,Clouds,mây đen u ám,21.2,21.86,95,1012,2.63,100,VN
CanTho,2025-05-25T21:20:13.664829Z,Cần Thơ,Rain,mưa nhẹ,25.01,26.18,100,1010,0.0,75,VN
Hue,2025-05-25T21:20:15.688052Z,Huế,Clouds,mây rải rác,27.05,30.77,89,1011,3.09,40,VN
HoChiMinh,2025-05-25T21:29:13.386561Z,Thành phố Hồ Chí Minh,Clouds,mây cụm,28.3,32.52,78,1010,3.6,75,VN
Hanoi,2025-05-25T21:29:15.575476Z,Xom Pho,Clouds,mây đen u ám,23.98,24.05,62,1016,5.32,87,VN
Danang,2025-05-25T21:29:17.839396Z,Ap Ba,Clouds,mây rải rác,28.99,35.61,84,1011,1.54,40,VN
GiaLai,2025-05-25T21:29:19.850229Z,Plây Cu,Clouds,mây đen u ám,21.2,21.86,95,1012,2.63,100,VN


In [0]:
df_silver.select("city_name", "location", "country_code").distinct().show(truncate=False)
print("Số dòng distinct:", df_silver.select("city_name", "location", "country_code").distinct().count())


+---------+---------------------+------------+
|city_name|location             |country_code|
+---------+---------------------+------------+
|CanTho   |Cần Thơ              |VN          |
|Hue      |Huế                  |VN          |
|Hanoi    |Xom Pho              |VN          |
|HoChiMinh|Thành phố Hồ Chí Minh|VN          |
|Danang   |Ap Ba                |VN          |
|GiaLai   |Plây Cu              |VN          |
+---------+---------------------+------------+

Số dòng distinct: 6


#### Thiết kế mô hình Data Warehouse lớp Gold


##### dim city

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
dim_city = (

    df_silver.select("city_name", "location", "country_code")   # lấy các cột liên quan đến thành phố
             .dropDuplicates()                                  # loại bỏ trùng lặp
             .withColumn("city_id", monotonically_increasing_id())  # tạo khóa chính surrogate key
)
dim_city.write.format("delta").mode("overwrite").save("/mnt/lakehouse1/gold/dim_city")

##### dim_condition

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
dim_condition = (
    df_silver.select("weather_main", "weather_description")  # lấy điều kiện thời tiết
             .dropDuplicates()                              # loại bỏ trùng lặp
             .withColumn("condition_id", monotonically_increasing_id())  # surrogate key
)
dim_condition.write.format("delta").mode("overwrite").save("/mnt/lakehouse1/gold/dim_weather_condition")


##### dim_time

In [0]:
from pyspark.sql.functions import hour, dayofmonth, month, year, date_format, minute
from pyspark.sql.functions import monotonically_increasing_id

dim_time = (
    df_silver.select("timestamp").dropDuplicates()
             .withColumn("time_id", monotonically_increasing_id())
             .withColumn("hour", hour("timestamp"))
             .withColumn("minute", minute("timestamp")) 
             .withColumn("day", dayofmonth("timestamp"))
             .withColumn("month", month("timestamp"))
             .withColumn("year", year("timestamp"))
             .withColumn("weekday", date_format("timestamp", "E"))
)
dim_time.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/mnt/lakehouse1/gold/dim_time")

##### fact_weather

In [0]:
fact_weather = (
    df_silver
    .join(dim_city, ["city_name", "location", "country_code"])                  # lấy city_id
    .join(dim_condition, ["weather_main", "weather_description"])              # lấy condition_id
    .join(dim_time, ["timestamp"])                                             # lấy time_id
    .select(
        "time_id", "city_id", "condition_id",     # khoá ngoại
        "temp", "feels_like", "humidity",         # chỉ số thời tiết
        "pressure", "wind_speed", "cloud_coverage"
    )
)
fact_weather.write.format("delta").mode("overwrite").save("/mnt/lakehouse1/gold/fact_weather")

##### Kiểm tra data sau khi lưu

In [0]:
spark.read.format("delta").load("/mnt/lakehouse1/gold/dim_city").show(truncate=False)

+---------+---------------------+------------+-------+
|city_name|location             |country_code|city_id|
+---------+---------------------+------------+-------+
|CanTho   |Cần Thơ              |VN          |0      |
|Hue      |Huế                  |VN          |1      |
|Hanoi    |Xom Pho              |VN          |2      |
|HoChiMinh|Thành phố Hồ Chí Minh|VN          |3      |
|Danang   |Ap Ba                |VN          |4      |
|GiaLai   |Plây Cu              |VN          |5      |
+---------+---------------------+------------+-------+



In [0]:
spark.read.format("delta").load("/mnt/lakehouse1/gold/dim_weather_condition").show(truncate=False)

+------------+-------------------+------------+
|weather_main|weather_description|condition_id|
+------------+-------------------+------------+
|Clouds      |mây thưa           |0           |
|Clouds      |mây rải rác        |1           |
|Clouds      |mây đen u ám       |2           |
|Clear       |bầu trời quang đãng|3           |
|Rain        |mưa nhẹ            |4           |
|Clouds      |mây cụm            |5           |
|Rain        |mưa vừa            |6           |
+------------+-------------------+------------+



In [0]:
spark.read.format("delta").load("/mnt/lakehouse1/gold/dim_time").show(truncate=False)

+--------------------------+-------+----+---+-----+----+-------+------+
|timestamp                 |time_id|hour|day|month|year|weekday|minute|
+--------------------------+-------+----+---+-----+----+-------+------+
|2025-05-24 20:38:55.637346|0      |20  |24 |5    |2025|Sat    |38    |
|2025-05-25 21:47:58.117252|1      |21  |25 |5    |2025|Sun    |47    |
|2025-05-25 21:29:15.575476|2      |21  |25 |5    |2025|Sun    |29    |
|2025-05-24 20:37:34.897362|3      |20  |24 |5    |2025|Sat    |37    |
|2025-05-24 20:38:10.102842|4      |20  |24 |5    |2025|Sat    |38    |
|2025-05-25 21:29:59.763628|5      |21  |25 |5    |2025|Sun    |29    |
|2025-05-24 21:08:30.862031|6      |21  |24 |5    |2025|Sat    |8     |
|2025-05-24 20:40:10.278945|7      |20  |24 |5    |2025|Sat    |40    |
|2025-05-25 21:46:04.596442|8      |21  |25 |5    |2025|Sun    |46    |
|2025-05-24 20:47:48.323391|9      |20  |24 |5    |2025|Sat    |47    |
|2025-05-25 21:34:46.554118|10     |21  |25 |5    |2025|Sun    |

In [0]:
spark.read.format("delta").load("/mnt/lakehouse1/gold/fact_weather").show(truncate=False)

+-------+-------+------------+-----+----------+--------+--------+----------+--------------+
|time_id|city_id|condition_id|temp |feels_like|humidity|pressure|wind_speed|cloud_coverage|
+-------+-------+------------+-----+----------+--------+--------+----------+--------------+
|487    |4      |0           |29.0 |35.64     |84      |1009    |3.09      |20            |
|391    |5      |5           |21.23|21.92     |96      |1011    |2.15      |74            |
|468    |0      |4           |26.01|26.01     |100     |1008    |1.03      |75            |
|557    |1      |0           |29.04|34.63     |79      |1009    |5.66      |20            |
|174    |3      |0           |27.18|30.36     |82      |1008    |1.54      |20            |
|270    |2      |2           |22.98|23.37     |78      |1013    |3.3       |100           |
|534    |4      |0           |29.0 |35.64     |84      |1009    |3.09      |20            |
|470    |5      |5           |21.23|21.92     |96      |1011    |2.15      |74  